In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [13]:
# Set vocabulary size and maximum review length
vocab_size = 20000  # Only keep the 10,000 most frequent words
max_length = 500    # Max length of reviews (truncate/pad)

# Load IMDB dataset (only keeping the most common words)
(train_sequences, train_labels), (test_sequences, test_labels) = imdb.load_data(num_words=vocab_size)

# Print sample review (as a sequence of integers)
print(f"First review (as sequence): {train_sequences[0]}")
print(f"Label: {train_labels[0]}")  # 1 = Positive, 0 = Negative


First review (as sequence): [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
Label: 1


In [14]:
# Pad sequences to ensure uniform input length
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

print(f"Shape of training data: {train_padded.shape}")
print(f"Shape of test data: {test_padded.shape}")


Shape of training data: (25000, 500)
Shape of test data: (25000, 500)


In [26]:
tf.keras.backend.clear_session()

# Build the RNN model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=265, input_length=max_length),  # Increased embedding size
    Bidirectional(LSTM(128, return_sequences=False)),  # LSTM for better sequence learning
    BatchNormalization(),  # Normalize activations
    Dense(64, activation="relu", kernel_initializer="glorot_uniform"),  # Better weight initialization
    Dropout(0.5),  # Dropout for regularization
    Dense(1, activation="sigmoid")  # Output Layer (Binary classification)
])

# Force the model to build
model.build(input_shape=(None, max_length))

# Compile the model with Adam optimizer and learning rate tuning
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])

# Print model summary
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 500, 265)            │       5,300,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ (None, 256)                 │         403,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,720,993 (21.82 MB)

 Trainable params: 5,720,481 (21.82 MB)

 Non-trainable params: 512 (2.00 KB)

In [27]:
history = model.fit(
    train_padded, train_labels,
    epochs=10,
    validation_data=(test_padded, test_labels),
    batch_size=64
)


Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 37s 85ms/step - accuracy: 0.6815 - loss: 0.5884 - val_accuracy: 0.7983 - val_loss: 0.4685
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 35s 91ms/step - accuracy: 0.8814 - loss: 0.2935 - val_accuracy: 0.8305 - val_loss: 0.4214
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 89ms/step - accuracy: 0.9349 - loss: 0.1761 - val_accuracy: 0.6009 - val_loss: 3.2570
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 40s 86ms/step - accuracy: 0.8497 - loss: 0.3343 - val_accuracy: 0.7959 - val_loss: 0.5144
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - accuracy: 0.9585 - loss: 0.1192 - val_accuracy: 0.8004 - val_loss: 0.7347
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 86ms/step - accuracy: 0.9810 - loss: 0.0546 - val_accuracy: 0.8020 - val_loss: 0.7563
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 42s 90ms/step - accuracy: 0.9910 - loss: 0.0278 - val_accuracy: 0.8254 - val_loss: 0.9022
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 41s 90ms/step - accuracy: 0.9937 - loss: 0.0193 - 

In [28]:
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print(f"Test Accuracy: {test_acc:.4f}")


782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.8249 - loss: 1.0792
Test Accuracy: 0.8247


In [41]:
# Adjust word index to align with dataset
word_index = {k: (v + 3) for k, v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

# Function to encode a new review
def encode_review(review, word_index, max_length):
    words = review.lower().split()  # Convert to lowercase and split into words
    encoded = [word_index.get(word, 2) for word in words]  # Use 2 for unknown words
    padded = pad_sequences([encoded], maxlen=max_length, padding="post")
    return padded

# Example review
sample_review = "This movie was fantastic! The story and acting were great."
sample_padded = encode_review(sample_review, word_index, max_length)

# Predict sentiment
prediction = model.predict(sample_padded)
sentiment = "Positive" if prediction[0] > 0.5 else "Negative"
print(f"Predicted Sentiment: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted Sentiment: Positive
